# Store music metadata using apple shortcuts
In this small project I was trying to store the metadata about the music that gets shazamed on my macOS
I started by trying to find Shazam database file on my system but that proved to be quit tricky
Note that I tried this on MacOs, it might be different in iOS device
TODO: test on mobile device

## Create a Shortcut
the the shortcut app, click on the plus sign, a new shortcut will be created
<img src="imgs/shazam_create.png" alt="create a new shortcut" width="70%" style="margin:5px;"/>


also note that in the video I recorded I used a loop, bad idea since we can run a shortcut in CLI

I modified the short cut to perform one cycle, it takes no input and outputs a song metadata if a song is found as text, otherwise it return some "No song found"

<img src="imgs/shortcut.png" alt="shows the text file created by the shortcut app" width="80%" style="margin:5px;"/>

In [1]:
import os
import re
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
SHAZAM_TEMPLATE = """
<?xml version="1.0" encoding="UTF-8"?>
<root>

<timestamp>
Date
</timestamp>

<title>
Shazam Media (Title)
</title>

<artist>
Shazam Media (Artist)
</artist>

<isexplicit>
Shazam Media (Is Explicit)
</isexplicit>

<lyricssnippet>
Shazam Media (Lyrics Snippet)
</lyricssnippet>

<lyricsnippetsynced>
Shazam Media (Lyric Snippet Synced)
</lyricsnippetsynced>

<artwork>
Shazam Media (Artwork)
</artwork>

<videourl>
Shazam Media (Video URL)
</videourl>

<shazamurl>
Shazam Media (Shazam URL)
</shazamurl>

<applemusicurl>
Shazam Media (Apple Music URL)
</applemusicurl>

<name>
Shazam Media (Name)
</name>

</root>
"""

In [3]:
outfile = './test.txt'
res = !shortcuts run "shazam_step" --output-path $outfile

In [4]:
res

[]

find the valid tags then find their values

In [5]:
def parse_row(outfile, encoding='utf-8'):
    """
    The parse_row function parses the XML document using lxml parser.
    it uses the `SHAZAM_TEMPLATE` string as map to read the xml data
    ---------------------------------------------------------
    :param outfile: Specify the path to the xml file that will be parsed
    :return: A dictionary with keys as the tag names
        and values as the text content of those tags
    """
    soup = BeautifulSoup(SHAZAM_TEMPLATE, 'xml')
    root = soup.root
    tags = [tag.name  for tag in root.children if tag and tag.name!='root']
    print()
    try:
        with open(outfile, encoding=encoding) as f:
            row = f.read()
    except FileNotFoundError:
        return FileNotFoundError
    # print(row)
    soup = BeautifulSoup(row.replace('\n', ''), 'xml')
    root = soup.select_one('root')
    if root:
        dct = {}
        for tag in tags:
            if not tag:
                continue
            tag_content = root.find(tag)
            if tag_content:
                dct[tag] = tag_content.text
    return dct

In [6]:
parse_row(outfile)

{'timestamp': '10 May 2024 at 15:14',
 'title': 'Something In the Rain (Something In the Rain, Pt. 1 (Music from the Original TV Series))',
 'artist': 'Rachael Yamagata',
 'isexplicit': 'No',
 'lyricssnippet': 'I feel you now from hereOur time is not done yetYou’ve given me my fullest heartWe’ll never be apartI’m breathing through you now',
 'lyricsnippetsynced': 'I’ve never been so exposedAre there pieces still yet to beDiscovered with your love?I can see anything through wherever you areI can feel you',
 'artwork': 'Image',
 'videourl': '',
 'shazamurl': 'https://www.shazam.com/track/414805857/%EB%B0%A5-%EC%9E%98-%EC%82%AC%EC%A3%BC%EB%8A%94-%EC%98%88%EC%81%9C-%EB%88%84%EB%82%98-pt-1-music-from-the-original-tv-series?co=GB=shortcuts=181609=-2.1648407E-4=299453=2024-05-10T14:14:07.125Z',
 'applemusicurl': 'https://music.apple.com/gb/album/something-in-the-rain-something-in-the-rain-pt-1/1458350926?i=1458350930=30201=Shazam_shortcuts',
 'name': 'Rachael Yamagata - Something In the Rain 

In [7]:
e= Exception
type(e)

type

In [8]:
import os
import pandas as pd

WRITE_EXT = {
'.csv': 'to_csv',
'.xls': 'to_excel',
'.xlsx': 'to_excel',
'.json': 'to_json',
'.html': 'to_html',
'.sql': 'to_sql',
'.parquet': 'to_parquet',
'.feather': 'to_feather',
'.h5': 'to_hdf',
'.hdf': 'to_hdf',
'.dta': 'to_stata',
'.sas7bdat': 'to_sas',
}

READ_EXT = {
'.csv': 'read_csv',
'.xls': 'read_excel',
'.xlsx': 'read_excel',
'.json': 'read_json',
'.html': 'read_html',
'.sql': 'read_sql',
'.parquet': 'read_parquet',
'.feather': 'read_feather',
'.h5': 'read_hdf',
'.hdf': 'read_hdf',
'.dta': 'read_stata',
'.sas7bdat': 'read_sas',
}
def read(db_file, encoding='utf-8'):
  """
  The read function reads a database file and returns a pandas dataframe.
  -------------------------------------------
  :param db_file: Specify the file to be read
  :param encoding: Specify the encoding of the file
  :return: A pandas dataframe with the contents of the file
  """
  print(db_file)
  name, ext = os.path.splitext(db_file)
  name = name.split('/')[-1]
  method = READ_EXT.get(ext)
  if method:
      read_method = getattr(pd, method)
      df = read_method(db_file, encoding=encoding)
      return df

In [11]:
# x = read('file.json')
# x
import magic
# pip install python-magic
# brew install libmagic


def detect_file_type(file_path):
    """
    Detect the type of data in a file based on its content.

    Parameters:
    - file_path (str): Path to the file.

    Returns:
    - str: Detected file type.
    """
    # Create magic object
    detector = magic.Magic()

    # Detect file type
    file_type = detector.from_file(file_path)

    return file_type

In [14]:
import subprocess

def detect_file_type(file_path):
    """
    Detect the type of data in a file based on its content using the file command.

    Parameters:
    - file_path (str): Path to the file.

    Returns:
    - str: Detected file type.
    """
    # Run the file command and capture the output
    result = subprocess.run(['file', '--mime-type', file_path], capture_output=True, text=True)
    
    # Extract the file type from the output
    file_type = result.stdout.strip().split(': ')[-1]

    return file_type

# Example usage:
file_path = 'ww.txt'
file_type = detect_file_type(file_path)
print("File Type:", file_type)

File Type: text/xml
